In [52]:
pip install  streamlit joblib


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [88]:
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import numpy as np
import joblib

# Load trained model and encoders
model = joblib.load("Car Prediction Model.pkl")
make_encoder = joblib.load("make_encoder.pkl")
model_encoder = joblib.load("model_encoder.pkl")
condition_encoder = joblib.load("condition_encoder.pkl")

# Get original categories
make_categories = list(make_encoder.classes_)
model_categories = list(model_encoder.classes_)
condition_categories = list(condition_encoder.classes_)

# Streamlit UI
st.title("Car Price Prediction")
st.write("Enter car details to predict the price.")

# Input fields
make = st.selectbox("Select Make", make_categories)
model_name = st.selectbox("Select Model", model_categories)
year = st.number_input("Enter Year", min_value=2000, max_value=2025, value=2015)
mileage = st.number_input("Enter Mileage", min_value=0, max_value=300000, value=50000)
condition = st.selectbox("Select Condition", condition_categories)

if st.button("Predict Price"):
    # Encode categorical inputs using trained encoders
    make_encoded = make_encoder.transform([make])[0]
    model_encoded = model_encoder.transform([model_name])[0]
    condition_encoded = condition_encoder.transform([condition])[0]

    # Create input array
    input_features = np.array([[make_encoded, model_encoded, year, mileage, condition_encoded]])

    # Ensure input shape matches training data
    st.write(f"Encoded Input Features: {input_features}")

    # Predict price
    price = model.predict(input_features)
    st.write(f"Predicted Price: ${price[0]:,.2f}")
""")


In [90]:
import streamlit as st
import numpy as np
import joblib

# Load trained model and encoders
model = joblib.load("Car Prediction Model.pkl")
make_encoder = joblib.load("make_encoder.pkl")
model_encoder = joblib.load("model_encoder.pkl")
condition_encoder = joblib.load("condition_encoder.pkl")

# Get original categories
make_categories = list(make_encoder.classes_)
model_categories = list(model_encoder.classes_)
condition_categories = list(condition_encoder.classes_)

# Streamlit UI
st.title("Car Price Prediction")
st.write("Enter car details to predict the price.")

# Input fields
make = st.selectbox("Select Make", make_categories)
model_name = st.selectbox("Select Model", model_categories)
year = st.number_input("Enter Year", min_value=2000, max_value=2025, value=2015)
mileage = st.number_input("Enter Mileage", min_value=0, max_value=300000, value=50000)
condition = st.selectbox("Select Condition", condition_categories)

if st.button("Predict Price"):
    # Encode categorical inputs using trained encoders
    make_encoded = make_encoder.transform([make])[0]
    model_encoded = model_encoder.transform([model_name])[0]
    condition_encoded = condition_encoder.transform([condition])[0]

    # Create input array
    input_features = np.array([[make_encoded, model_encoded, year, mileage, condition_encoded]])

    # Ensure input shape matches training data
    st.write(f"Encoded Input Features: {input_features}")

    # Predict price
    price = model.predict(input_features)
    st.write(f"Predicted Price: ${price[0]:,.2f}")


In [ ]:
!streamlit run app.py	
